In [1]:
import pandas as pd
import pickle
import string
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import re
import numpy as np
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import re 
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to /home/dval/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('kp_tweets_cleaned.pickle', "rb") as f:
    tweets = pickle.load(f)

In [25]:
#Gather twitter-labeled english tweets and delete from rest of dataset
twitter_english = tweets[tweets["tweet_lang"]=="en"]
tweets = tweets[tweets["tweet_lang"] != "en"]

In [27]:
#Import nltk stopwords and search for additional english tweets using stop words
stop_words = set(stopwords.words('english'))
english_stopwords = list(stop_words)
def exact_match(stopwords, string):
    for word in stopwords:
        if re.search(r'\b' + word + r'\b', string):
            return True
        else:
            return False
tweets["english"] = tweets.clean_text.apply(lambda x: exact_match(english_stopwords, x))
english_from_stopwords = tweets[tweets["english"]==True]

In [33]:
#Combine both sets of english tweets
english_tweets = pd.concat([twitter_english, english_from_stopwords], ignore_index=True)

/home/dval/.conda/envs/symbolic/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [16]:
#Drop any remaining english language tweets from dataset and save english tweets
tweets = tweets[tweets["english"]==False]

with open("kp_tweets_cleaned_english.pickle", "wb") as f:
    pickle.dump(english_tweets, f)

In [5]:
#Gather twitter-identified urdu tweets from data and delete from dataset
twitter_urdu = tweets[tweets["tweet_lang"]=="ur"]
tweets = tweets[tweets["tweet_lang"]!="ur"]

In [20]:
#Upload user-created urdu stopwords and search for them in remaining tweets
stop_words = pd.read_excel("urdu_stopwords.xlsx")
urdu_stopwords = list(stop_words)
tweets["urdu"] = tweets.clean_text.apply(lambda x: exact_match(urdu_stopwords, x))
urdu_from_stopwords = tweets[tweets["urdu"]==True]

In [26]:
#Get wrongly categorized urdu tweets from data
urdu_wrong_cat = tweets[tweets["tweet_lang"]=="fa"]

In [27]:
#Concatenate all urdu-language tweets and export
urdu = pd.concat([twitter_urdu, urdu_from_stopwords, urdu_wrong_cat], ignore_index=True)
with open("kp_tweets_cleaned_urdu.pickle", "wb") as f:
    pickle.dump(twitter_urdu, f)

/home/dval/.conda/envs/symbolic/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
#Gather twitter identified pashto tweets and remove from dataset
twitter_pashto = tweets[tweets["tweet_lang"]=="ps"]
tweets = tweets[tweets["tweet_lang"]!="ps"]

Using Pashto Arabic Stopwords from #https://github.com/mohbadar/pashto-stopwords/blob/master/stopwords/pashto-stopwords.csv

In [10]:
#Gather more pashto tweets from stopword list (from url above) and remove from dataset
pashto_stopwords = pd.read_excel("new_pashto_stopwords.xlsx")
pashto_stopwords = pashto_stopwords["word"].to_list()
tweets["pashto"] = tweets.clean_text.apply(lambda x: exact_match(pashto_stopwords, x))
pashto_from_stopwords = tweets[tweets["pashto"]==True]
tweets = tweets[tweets["pashto"]==False]

In [18]:
#Concatente and save pashto language tweets
pashto_tweets = pd.concat([twitter_pashto, pashto_from_stopwords], ignore_index=True)
with open("kp_tweets_pashto.pickle", "wb") as f:
    pickle.dump(pashto_tweets, f)

/home/dval/.conda/envs/symbolic/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [39]:
#Group blank tweets or tweets with fewer than three words - save group of tweets with no or few words and no hashtags
tweets["clean_text"] = tweets["clean_text"].replace(r'^\s*$', np.nan, regex=True)
tweets["text_null"] = tweets["clean_text"].apply(lambda x: pd.isna(x))
tweets["hashtags_null"] = tweets["hashtags"].apply(lambda x: pd.isna(x))
blank_tweets = tweets[(tweets.text_null == True) & (tweets.hashtags_null == True)]
tweets = tweets[(tweets.text_null == False) | (tweets.hashtags_null == False)]
tweets["clean_text"] = tweets.clean_text.apply(lambda x: str(x))
tweets["clean_text"] = tweets.clean_text.apply(lambda x: x.lstrip(' '))
tweets["clean_text"] = tweets.clean_text.apply(lambda x: x.rstrip(' '))
tweets['number_of_words'] = tweets['clean_text'].str.count(' ') + 1
tweets["few_words"] = tweets["number_of_words"] < 3
few_words_no_hashtag = tweets[(tweets.few_words == True) & (tweets.hashtags_null == True)]
tweets = tweets[(tweets.few_words == False) | (tweets.hashtags_null == False)]
low_data_tweets = pd.concat([blank_tweets, few_words_no_hashtag], ignore_index=True)
with open("kp_tweets_no_data.pickle", "wb") as f:
    pickle.dump(low_data_tweets, f)
    

In [48]:
#Gather and save blank tweets or tweets with fewer than three words that DO have hashtags
hashtags_only1 = tweets[(tweets.text_null == True) & (tweets.hashtags_null == False)]
hashtags_only2 = tweets[(tweets.few_words == True) & (tweets.hashtags_null == False)]
hashtags_only = pd.concat([hashtags_only1, hashtags_only2], ignore_index=True)
with open("kp_tweets_hashtags_only.pickle", "wb") as f:
    pickle.dump(hashtags_only, f)

/home/dval/.conda/envs/symbolic/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [61]:
#Remove these low-data tweets from main dataset
tweets = tweets.drop(tweets[(tweets.text_null == True) & (tweets.hashtags_null == False)].index)
tweets = tweets.drop(tweets[(tweets.few_words == True) & (tweets.hashtags_null == False)].index)

In [64]:
#Gather for romanized urdu tweets using user-created stopword list and remove from dataframe
rus = pd.read_excel("romanized_urdu_stopwords.xlsx")
rus = rus["romanized_urdu"].to_list()
tweets["romanized_urdu"] = tweets.clean_text.apply(lambda x: exact_match(rus, x))
romanized_urdu = tweets[tweets["romanized_urdu"]==True]
tweets = tweets[tweets["romanized_urdu"]==False]

In [71]:
#Gather wrongly categorized romanized urdu tweets
ru1 = tweets[tweets["tweet_lang"]=="in"]
ru2 = tweets[tweets["tweet_lang"]=="hi"]
ru3 = tweets[tweets["tweet_lang"]=="tl"]
ru4 = tweets[tweets["tweet_lang"]=="et"]
ru5 = tweets[tweets["tweet_lang"]=="ht"]
ru6 = tweets[tweets["tweet_lang"]=="id"]
ru7 = tweets[tweets["tweet_lang"]=="fi"]

In [78]:
#Remove wrongly categorized romanized urdu tweets from dataframe
tweets = tweets[tweets["tweet_lang"]!="in"]
tweets = tweets[tweets["tweet_lang"]!="hi"]
tweets = tweets[tweets["tweet_lang"]!="tl"]
tweets = tweets[tweets["tweet_lang"]!="et"]
tweets = tweets[tweets["tweet_lang"]!="ht"]
tweets = tweets[tweets["tweet_lang"]!="id"]
tweets = tweets[tweets["tweet_lang"]!="fi"]

In [79]:
#Concat and save romanized urdu tweets
romanized_urdu = pd.concat([romanized_urdu, ru1, ru2, ru3, ru4, ru5, ru6, ru7], ignore_index=True)
with open("kp_tweets_romanized_urdu.pickle", "wb") as f:
    pickle.dump(romanized_urdu, f)

In [396]:
#Gather romanized pashto tweets using user-created stopword list and remove from dataframe and save
rps = pd.read_excel("romanized_pashto_stopwords.xlsx")
rps = rps["romanized_pashto"].to_list()
tweets["romanized_pashto"] = tweets.clean_text.apply(lambda x: exact_match(rps, x))
romanized_pashto = tweets[tweets["romanized_pashto"]==True]
tweets = tweets[tweets["romanized_pashto"]==False]
with open("kp_tweets_romanized_pashto.pickle", "wb") as f:
    pickle.dump(romanized_pashto, f)

In [474]:
#Save all tweets in other unidentified languages
with open("kp_tweets_other_languages.pickle", "wb") as f:
    pickle.dump(tweets, f)